# RAG Application with Knowledge Graphs for SEC Filings

## Import packages and set-up Neo4j

In [1]:
from dotenv import load_dotenv
import os

In [2]:
import json
import textwrap

In [3]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

In [4]:
import warnings
warnings.filterwarnings("ignore")

Load API information for OpenAI and Neo4j from `.env` file

In [5]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
AURA_INSTANCEID = os.getenv('AURA_INSTANCEID')
AURA_INSTANCENAME = os.getenv('AURA_INSTANCENAME')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [6]:
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

__Form 10-K JSON file__

Working with a single 10-K form for a company called NetApp.  
The data from 10-k form has already been extracted, cleaned and structured in the form of a JSON.  
Data is taken from the __'Knowledge Graphs for RAG'__ course by __DeepLearning.AI__. 

In [7]:
form_10k_file_name = "data/0000950170-23-027948.json"
form_10k_object = json.load(open(form_10k_file_name))

In [8]:
for k, v in form_10k_object.items():
    print(k, type(v))

item1 <class 'str'>
item1a <class 'str'>
item7 <class 'str'>
item7a <class 'str'>
cik <class 'str'>
cusip6 <class 'str'>
cusip <class 'list'>
names <class 'list'>
source <class 'str'>


In [9]:
item1_text = form_10k_object['item1']

In [10]:
item1_text[:2000]

'>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the complexities created by rapid data and cloud growth, multi-cloud management, and the adoption of next-generation technologies, such as AI, Kubernetes, and modern databases. Our modern approach to hybrid, multicloud infrastructure and data management, which we term ‘evolved cloud’, provi

__Split Form 10-K sections into chunks__

Set up text splitter using LangChain

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 200,
    length_function = len,
    is_separator_regex = False
)

In [12]:
item1_text_chunks = text_splitter.split_text(item1_text)
type(item1_text_chunks)

list

In [13]:
len(item1_text_chunks)

254

In [14]:
item1_text_chunks[5]

'Cloud storage, data services, and software\n\n\nThe NetApp Cloud Volumes Platform is an integrated collection of cloud storage infrastructure and data services. The platform is anchored by \nNetApp Cloud Volumes ONTAP\n, a cloud-based software for customers who wish to manage their own cloud storage infrastructure. It is based on the same ONTAP data management software that underpins our storage infrastructure offerings. Fully managed cloud storage offerings are available natively on Microsoft Azure as \nAzure NetApp Files\n, on AWS as \nAmazon FSx for NetApp ONTAP\n, and on Google Cloud as \nNetApp Cloud Volumes Service for Google Cloud.\n \n\n\nManageability\n\n\nAt the heart of our public cloud storage and data service offerings is \nNetApp\n \nBlueXP\n. BlueXP is a unified control plane that enables customers to manage their entire data landscape through one single, SaaS-delivered point of control. NetApp BlueXP combines storage and data services via its unified control plane to c

__Helper function to chunk all sections of the Form 10-K__

In [15]:
def split_form10k_data_from_file(file):
    chunks_with_metadata = [] # use this to accumlate chunk records
    file_as_object = json.load(open(file)) # open the json file
    for item in ['item1', 'item1a', 'item7', 'item7a']: # pull these keys from the json
        print(f'Processing {item} from {file}')
        item_text = file_as_object[item] # grab the text of the item
        item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
        chunk_seq_id = 0
        for chunk in item_text_chunks[:20]: # only take the first 20 chunks
            form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
            # finally, construct a record with metadata and the chunk text
            chunks_with_metadata.append({
                'text': chunk,
                # metadata from looping...
                'f10kItem': item,
                'chunkSeqId': chunk_seq_id,
                # constructed metadata...
                'formId': f'{form_id}', # pulled from the filename
                'chunkId': f'{form_id}-{item}-chunk{chunk_seq_id:04d}',
                # metadata from file...
                'names': file_as_object['names'],
                'cik': file_as_object['cik'],
                'cusip6': file_as_object['cusip6'],
                'source': file_as_object['source'],
            })
            chunk_seq_id += 1
        print(f'\tSplit into {chunk_seq_id} chunks')
    return chunks_with_metadata

In [16]:
form10k_file_chunks = split_form10k_data_from_file(form_10k_file_name)

Processing item1 from data/0000950170-23-027948.json
	Split into 20 chunks
Processing item1a from data/0000950170-23-027948.json
	Split into 1 chunks
Processing item7 from data/0000950170-23-027948.json
	Split into 1 chunks
Processing item7a from data/0000950170-23-027948.json
	Split into 1 chunks


In [17]:
form10k_file_chunks[0]

{'text': '>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the complexities created by rapid data and cloud growth, multi-cloud management, and the adoption of next-generation technologies, such as AI, Kubernetes, and modern databases. Our modern approach to hybrid, multicloud infrastructure and data management, which we term ‘evolved clou

__Create graph nodes using text chunks__

In [18]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET
        mergedChunk.names = $chunkParam.names,
        mergedChunk.formId = $chunkParam.formId,
        mergedChunk.cik = $chunkParam.cik,
        mergedChunk.cusip6 = $chunkParam.cusip6,
        mergedChunk.source = $chunkParam.source,
        mergedChunk.f10kItem = $chunkParam.f10kItem,
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId,
        mergedChunk.text = $chunkParam.text
RETURN mergedChunk
"""

__Set up connection to graph instance using LangChain__

In [19]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
)

__Test single chunk node__

In [20]:
kg.query(merge_chunk_node_query, 
         params={'chunkParam':form10k_file_chunks[0]}
        )

[{'mergedChunk': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'names': ['Netapp Inc', 'NETAPP INC'],
   'cik': '1002047',
   'cusip6': '64110D',
   'source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm',
   'text': '>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the 

__Uniqueness constraint to avoid duplicate chunks__

In [21]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
"""
)

[]

In [22]:
kg.query("""
SHOW INDEXES
"""
)

[{'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 2,
  'name': 'unique_chunk',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['chunkId'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'unique_chunk',
  'lastRead': None,
  'readCount': 0}]

__Loop through and create nodes for all chunks__

In [23]:
node_count = 0
for chunk in form10k_file_chunks:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunkId']}")
    kg.query(merge_chunk_node_query,
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0000
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0001
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0002
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0003
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0004
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0005
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0006
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0007
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0008
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0009
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0010
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0011
Creating `:Chunk` node for chunk ID 0000950170-23-027948-item1-chunk0012
Creating `:Chunk` node for chunk ID 0000950170-23-0

In [24]:
kg.query("""
    MATCH(n)
    RETURN count(n) as nodeCount
    """)

[{'nodeCount': 23}]

__Create a vector index__

In [25]:
kg.query("""
    CREATE VECTOR INDEX `form_10k_chunks` IF NOT EXISTS
        FOR (c:Chunk) ON (c.textEmbedding)
        OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'
        }}
    """
)

[]

In [26]:
kg.query("""
SHOW INDEXES
"""
)

[{'id': 4,
  'name': 'form_10k_chunks',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['textEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': None},
 {'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 2,
  'name': 'unique_chunk',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entit

__Calculate embedding vectors for chunks and populate index__

In [27]:
kg.query("""
    MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
    WITH chunk, genai.vector.encode(
        chunk.text,
        "OpenAI",
        {token: $openAiApiKey}
    ) AS vector
    CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
""",
    params={"openAiApiKey":OPENAI_API_KEY}
)

[]

In [28]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Chunk {chunkId: STRING, names: LIST, formId: STRING, cik: STRING, cusip6: STRING, source: STRING, f10kItem: STRING, chunkSeqId: INTEGER, text: STRING, textEmbedding: LIST}
Relationship properties:

The relationships:



__Similarity search to find relevant chunks__

In [29]:
def neo4j_vector_search(question):
    """Search for similar nodes using the Neo4j vector index"""
    vector_search_query = """
        WITH genai.vector.encode($question, "OpenAI", {token: $openAiApiKey}) AS question_embedding
        CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
        RETURN score, node.text AS text
    """
    similar = kg.query(vector_search_query,
                      params={
                          'question': question,
                          'openAiApiKey': OPENAI_API_KEY,
                          'index_name': VECTOR_INDEX_NAME,
                          'top_k': 10
                      })
    return similar

In [30]:
search_results = neo4j_vector_search(
    'In a single sentence, tell me about Netapp.'
)

In [31]:
search_results[0]

{'score': 0.9356337785720825,
 'text': '>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the complexities created by rapid data and cloud growth, multi-cloud management, and the adoption of next-generation technologies, such as AI, Kubernetes, and modern databases. Our modern approach to hybrid, multicloud infrastructure and data managemen

### Setup LangChain RAG workflow to chat with the form

In [32]:
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 67, offset: 66} for query: "UNWIND $data AS row MATCH (n:`Chunk`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'textEmbedding', row.embedding) YIELD node RETURN count(*)"


In [33]:
retriever = neo4j_vector_store.as_retriever()

In [34]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever
)

In [35]:
def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain({"question": question},
                    return_only_outputs=True,)
    print(textwrap.fill(response['answer'], 60))

In [36]:
prettychain("What is Netapp's primary business?")

NetApp's primary business is enterprise storage and data
management, cloud storage, and cloud operations.


In [37]:
prettychain("""
    Tell me about Netapp. 
    Limit your answer to a single sentence.
""")

NetApp is a global cloud-led, data-centric software company
that provides customers with the freedom to manage
applications and data across hybrid multicloud environments.


In [38]:
prettychain("""
    Tell me about Apple. 
    Limit your answer to a single sentence.
""")

Apple is a global cloud-led, data-centric software company
headquartered in San Jose, California, that provides
customers with the freedom to manage applications and data
across hybrid multicloud environments.


In [39]:
prettychain("""
    Tell me about Apple. 
    Limit your answer to a single sentence.
    If you are unsure about the answer, say you don't know.
""")

I don't know.


## Adding Relationships to the Knowledge Graph

__Creating a Form 10-K node__

In [40]:
cypher = """
    MATCH (anyChunk:Chunk)
    WITH anyChunk LIMIT 1
    RETURN anyChunk { .name, .source, .formId, .cik, .cusip6 } as formInfo
"""
form_info_list = kg.query(cypher)
form_info_list

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 4, column: 24, offset: 77} for query: '\n    MATCH (anyChunk:Chunk)\n    WITH anyChunk LIMIT 1\n    RETURN anyChunk { .name, .source, .formId, .cik, .cusip6 } as formInfo\n'


[{'formInfo': {'cik': '1002047',
   'source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm',
   'formId': '0000950170-23-027948',
   'name': None,
   'cusip6': '64110D'}}]

In [41]:
form_info = form_info_list[0]['formInfo']

In [42]:
cypher = """
    MERGE (f:Form {formId: $formInfoParam.formId })
    ON CREATE
        SET f.names = $formInfoParam.names
        SET f.source = $formInfoParam.source
        SET f.cik = $formInfoParam.cik
        SET f.cusip6 = $formInfoParam.cusip6
"""

kg.query(cypher, params={'formInfoParam': form_info})

[]

In [43]:
kg.query("MATCH (f:Form) RETURN count(f) as formCount")

[{'formCount': 1}]

__Create a linked list of Chunk nodes for each section__

In [44]:
cypher = """
    MATCH (from_same_form:Chunk)
        WHERE from_same_form.formId = $formIdParam
    RETURN from_same_form {.formId, .f10kItem, .chunkId, .chunkSeqId } as chunkInfo
        LIMIT 10
"""

kg.query(cypher, params={'formIdParam': form_info['formId']})

[{'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0001',
   'chunkSeqId': 1}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0002',
   'chunkSeqId': 2}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0003',
   'chunkSeqId': 3}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0004',
   'chunkSeqId': 4}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0005',
   'chunkSeqId': 5}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-

__Order chunks by their sequence ID__

In [45]:
cypher = """
    MATCH (from_same_form:Chunk)
        WHERE from_same_form.formId = $formIdParam
    RETURN from_same_form {.formId, .f10kItem, .chunkId, .chunkSeqId } as chunkInfo
        ORDER BY from_same_form.chunkSeqId ASC
        LIMIT 10
"""

kg.query(cypher, params={'formIdParam': form_info['formId']})

[{'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item7a',
   'chunkId': '0000950170-23-027948-item7a-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item7',
   'chunkId': '0000950170-23-027948-item7-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1a',
   'chunkId': '0000950170-23-027948-item1a-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0001',
   'chunkSeqId': 1}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0002',
   'chunkSeqId': 2}},
 {'chunkInfo': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950

In [46]:
cypher = """
    MATCH (from_same_section:Chunk)
    WHERE from_same_section.formId = $formIdParam
        AND from_same_section.f10kItem = $f10kItemParam // NEW!!!
    RETURN from_same_section { .formId, .f10kItem, .chunkId, .chunkSeqId }
        ORDER BY from_same_section.chunkSeqId ASC
        LIMIT 10
"""

kg.query(cypher, params={'formIdParam': form_info['formId'], 'f10kItemParam': 'item1'})

[{'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0000',
   'chunkSeqId': 0}},
 {'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0001',
   'chunkSeqId': 1}},
 {'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0002',
   'chunkSeqId': 2}},
 {'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0003',
   'chunkSeqId': 3}},
 {'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0004',
   'chunkSeqId': 4}},
 {'from_same_section': {'formId': '0000950170-23-027948',
   'f10kItem': 'item1',
   'chunkId': '0000950170-23-027948-item1-chunk0005',
   'chunkSeqId': 5}},
 {'from_same_section': {'formId': '0000950170-23-027

In [47]:
cypher = """
    MATCH (from_same_section:Chunk)
    WHERE from_same_section.formId = $formIdParam
        AND from_same_section.f10kItem = $f10kItemParam
    WITH from_same_section { .formId, .f10kItem, .chunkId, .chunkSeqId }
        ORDER BY from_same_section.chunkSeqId ASC
        LIMIT 10
    RETURN collect(from_same_section) // NEW!!
"""

kg.query(cypher, params={'formIdParam': form_info['formId'], 'f10kItemParam': 'item1'})

[{'collect(from_same_section)': [{'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0000',
    'chunkSeqId': 0},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0001',
    'chunkSeqId': 1},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0002',
    'chunkSeqId': 2},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0003',
    'chunkSeqId': 3},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0004',
    'chunkSeqId': 4},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0005',
    'chunkSeqId': 5},
   {'formId': '0000950170-23-027948',
    'f10kItem': 'item1',
    'chunkId': '0000950170-23-027948-item1-chunk0006',
    'chunkSe

__Add a NEXT relationship between subsequent chunks__

Use the `apoc.nodes.link` function from Neo4j to link ordered list of Chunk nodes with a `NEXT` relationship.  
__Test with "Item 1" section__

In [48]:
cypher = """
    MATCH (from_same_section:Chunk)
    WHERE from_same_section.formId = $formIdParam
        AND from_same_section.f10kItem = $f10kItemParam
    WITH from_same_section
        ORDER BY from_same_section.chunkSeqId ASC
    WITH collect(from_same_section) as section_chunk_list
        CALL apoc.nodes.link(
            section_chunk_list, 
            "NEXT", 
            {avoidDuplicates: true}
        )  // NEW!!!
    RETURN size(section_chunk_list)
"""

kg.query(cypher, params={'formIdParam': form_info['formId'], 
                         'f10kItemParam': 'item1'})

[{'size(section_chunk_list)': 20}]

In [49]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Chunk {chunkId: STRING, names: LIST, formId: STRING, cik: STRING, cusip6: STRING, source: STRING, f10kItem: STRING, chunkSeqId: INTEGER, text: STRING, textEmbedding: LIST}
Form {formId: STRING, cik: STRING, cusip6: STRING, source: STRING}
Relationship properties:

The relationships:
(:Chunk)-[:NEXT]->(:Chunk)


__Loop through and create relationships for all sections of the form 10-K__

In [50]:
cypher = """
    MATCH (from_same_section:Chunk)
    WHERE from_same_section.formId = $formIdParam
        AND from_same_section.f10kItem = $f10kItemParam
    WITH from_same_section
        ORDER BY from_same_section.chunkSeqId ASC
    WITH collect(from_same_section) as section_chunk_list
        CALL apoc.nodes.link(
            section_chunk_list, 
            "NEXT", 
            {avoidDuplicates: true}
        )
    RETURN size(section_chunk_list)
"""
for form10kItemName in ['item1', 'item1a', 'item7', 'item7a']:
    kg.query(cypher, params={'formIdParam':form_info['formId'], 
                           'f10kItemParam': form10kItemName})


__Connect chunks to their parent form with a PART_OF relationship__

In [51]:
cypher = """
    MATCH (c:Chunk), (f:Form)
        WHERE c.formId = f.formId
    MERGE (c)-[newRelationship:PART_OF]->(f)
    RETURN count(newRelationship)
"""

kg.query(cypher)

[{'count(newRelationship)': 23}]

__Create a SECTION relationship on first chunk of each section__

In [52]:
cypher = """
    MATCH (first:Chunk), (f:Form)
    WHERE first.formId = f.formId
        AND first.chunkSeqId = 0
    WITH first, f
        MERGE (f)-[r:SECTION {f10kItem: first.f10kItem}]->(first)
    RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 4}]

__Example cypher queries__

In [53]:
cypher = """
MATCH (f:Form)-[r:SECTION]->(first:Chunk)
    WHERE f.formId = $formIdParam
        AND r.f10kItem = $f10kItemParam
RETURN first.chunkId as chunkId, first.text as text
"""

first_chunk_info = kg.query(cypher, params={
    'formIdParam': form_info['formId'], 
    'f10kItemParam': 'item1'
})[0]

first_chunk_info

{'chunkId': '0000950170-23-027948-item1-chunk0000',
 'text': '>Item 1.  \nBusiness\n\n\nOverview\n\n\nNetApp, Inc. (NetApp, we, us or the Company) is a global cloud-led, data-centric software company. We were incorporated in 1992 and are headquartered in San Jose, California. Building on more than three decades of innovation, we give customers the freedom to manage applications and data across hybrid multicloud environments. Our portfolio of cloud services, and storage infrastructure, powered by intelligent data management software, enables applications to run faster, more reliably, and more securely, all at a lower cost.\n\n\nOur opportunity is defined by the durable megatrends of data-driven digital and cloud transformations. NetApp helps organizations meet the complexities created by rapid data and cloud growth, multi-cloud management, and the adoption of next-generation technologies, such as AI, Kubernetes, and modern databases. Our modern approach to hybrid, multicloud infrastruct

In [54]:
cypher = """
  MATCH (first:Chunk)-[:NEXT]->(nextChunk:Chunk)
    WHERE first.chunkId = $chunkIdParam
  RETURN nextChunk.chunkId as chunkId, nextChunk.text as text
"""

next_chunk_info = kg.query(cypher, params={
    'chunkIdParam': first_chunk_info['chunkId']
})[0]

next_chunk_info

{'chunkId': '0000950170-23-027948-item1-chunk0001',
 'text': "•\nFlexibility and consistency: NetApp makes moving data and applications between environments seamless through a common storage foundation across on-premises and multicloud environments.\n\n\n•\nCyber resilience: NetApp unifies monitoring, data protection, security, governance, and compliance for total cyber resilience - with consistency and automation across environments. \n\n\n•\nContinuous operations: NetApp uses AI-driven automation for continuous optimization to service applications and store stateless and stateful applications at the lowest possible costs.\n\n\n•\nSustainability: NetApp has industry-leading tools to audit consumption, locate waste, and set guardrails to stop overprovisioning.\n\n\nProduct, Solutions and Services Portfolio\n \n\n\nNetApp's portfolio of cloud services and storage infrastructure is powered by intelligent data management software. Our operations are organized into two segments: Hybrid Clo

In [55]:
print(first_chunk_info['chunkId'], next_chunk_info['chunkId'])

0000950170-23-027948-item1-chunk0000 0000950170-23-027948-item1-chunk0001


In [56]:
cypher = """
    MATCH (c1:Chunk)-[:NEXT]->(c2:Chunk)-[:NEXT]->(c3:Chunk) 
        WHERE c2.chunkId = $chunkIdParam
    RETURN c1.chunkId, c2.chunkId, c3.chunkId
    """

kg.query(cypher,
         params={'chunkIdParam': next_chunk_info['chunkId']})

[{'c1.chunkId': '0000950170-23-027948-item1-chunk0000',
  'c2.chunkId': '0000950170-23-027948-item1-chunk0001',
  'c3.chunkId': '0000950170-23-027948-item1-chunk0002'}]

__Information is stored in the structure of a graph__  
- Matched patterns of nodes and relationships in a graph are called __*paths*__
- The length of a path is equal to the number of relationships in the path
- Paths can be captured as variables and used elsewhere in queries

In [57]:
cypher = """
    MATCH window = (c1:Chunk)-[:NEXT]->(c2:Chunk)-[:NEXT]->(c3:Chunk) 
        WHERE c1.chunkId = $chunkIdParam
    RETURN length(window) as windowPathLength
    """

kg.query(cypher,
         params={'chunkIdParam': next_chunk_info['chunkId']})

[{'windowPathLength': 2}]

__Finding variable length windows__

In [58]:
cypher = """
    MATCH window=(c1:Chunk)-[:NEXT]->(c2:Chunk)-[:NEXT]->(c3:Chunk) 
        WHERE c2.chunkId = $chunkIdParam
    RETURN nodes(window) as chunkList
    """
# pull the chunk ID from the first 
kg.query(cypher,
         params={'chunkIdParam': first_chunk_info['chunkId']})


[]

__Modify `NEXT` relationship to jave variable length__

In [59]:
cypher = """
  MATCH window=
      (:Chunk)-[:NEXT*0..1]->(c:Chunk)-[:NEXT*0..1]->(:Chunk) 
    WHERE c.chunkId = $chunkIdParam
  RETURN length(window)
  """

kg.query(cypher,
         params={'chunkIdParam': first_chunk_info['chunkId']})

[{'length(window)': 0}, {'length(window)': 1}]

__Retrieve only the longest path__

In [60]:
cypher = """
  MATCH window=
      (:Chunk)-[:NEXT*0..1]->(c:Chunk)-[:NEXT*0..1]->(:Chunk)
    WHERE c.chunkId = $chunkIdParam
  WITH window as longestChunkWindow 
      ORDER BY length(window) DESC LIMIT 1
  RETURN length(longestChunkWindow)
  """

kg.query(cypher,
         params={'chunkIdParam': first_chunk_info['chunkId']})

[{'length(longestChunkWindow)': 1}]

#### Expand context around a chunk using a window

__Create a regular vector store that retrieves a single node__

In [61]:
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
windowless_retriever = neo4j_vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
windowless_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=windowless_retriever
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 67, offset: 66} for query: "UNWIND $data AS row MATCH (n:`Chunk`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'textEmbedding', row.embedding) YIELD node RETURN count(*)"


__Next, define a window retrieval query to get consecutive chunks__

In [62]:
retrieval_query_window = """
MATCH window=
    (:Chunk)-[:NEXT*0..1]->(node)-[:NEXT*0..1]->(:Chunk)
WITH node, score, window as longestWindow 
  ORDER BY length(window) DESC LIMIT 1
WITH nodes(longestWindow) as chunkList, node, score
  UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node, score
RETURN apoc.text.join(textList, " \n ") as text,
    score,
    node {.source} AS metadata
"""

__Set up a QA chain that will use the window retrieval query__

In [63]:
vector_store_window = Neo4jVector.from_existing_index(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=retrieval_query_window, # NEW!!!
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain_window = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever_window
)

__Compare the two chains__

In [64]:
question = "In a single sentence, tell me about Netapp's business."

In [65]:
answer = windowless_chain(
    {"question": question},
    return_only_outputs=True,
)
print(textwrap.fill(answer["answer"]))

NetApp is a global cloud-led, data-centric software company that
provides customers the freedom to manage applications and data across
hybrid multicloud environments, focusing on enterprise storage and
data management, cloud storage, and cloud operations markets.


In [66]:
answer = chain_window(
    {"question": question},
    return_only_outputs=True,
)
print(textwrap.fill(answer["answer"]))

NetApp is a company that offers storage-as-a-service and global
support for continuous operation in complex environments, with a focus
on proactive and preemptive technology support for operational
continuity across the NetApp hybrid cloud.


## Expanding the SEC Knowledge Graph

__Read the collection of Form 13s__

In [67]:
import csv

all_form13s = []

with open('data/form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
      all_form13s.append(row)

In [68]:
all_form13s[0:5]

[{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
  'managerCik': '1000275',
  'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'managerName': 'Royal Bank of Canada',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '64395000000.0',
  'shares': '842850'},
 {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt',
  'managerCik': '1002784',
  'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805',
  'managerName': 'SHELTON CAPITAL MANAGEMENT',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '2989085000.0',
  'shares': '39124'},
 {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt',
  'managerCik': '1007280',
  'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215',
  'managerName': 'PUBLIC 

In [69]:
len(all_form13s)

561

### Create company nodes in the graph

In [70]:
# work with just the first form fow now
first_form13 = all_form13s[0]

cypher = """
MERGE (com:Company {cusip6: $cusip6})
  ON CREATE
    SET com.companyName = $companyName,
        com.cusip = $cusip
"""

kg.query(cypher, params={
    'cusip6':first_form13['cusip6'], 
    'companyName':first_form13['companyName'], 
    'cusip':first_form13['cusip'] 
})

[]

In [71]:
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

kg.query(cypher)

[{'com': {'cusip': '64110D104',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

__Update the company name to match Form 10-K__

In [72]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  RETURN com.companyName, form.names
"""

kg.query(cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': None}]

In [73]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""

kg.query(cypher)

[]

__Create a `FILED` relationship between the company and the Form-10K node__

In [74]:
kg.query("""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
""")

[]

### Create manager nodes

In [75]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

kg.query(cypher, params={'managerParam': first_form13})

[]

In [76]:
kg.query("""
  MATCH (mgr:Manager)
  RETURN mgr LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

__Create a uniquness constraint to avoid duplicate managers__

In [77]:
kg.query("""
CREATE CONSTRAINT unique_manager 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.managerCik IS UNIQUE
""")

[]

__Create a fulltext index of manager names to enable text search__

In [78]:
kg.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.managerName]
""")

[]

In [79]:
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames", 
      "royal bank") YIELD node, score
  RETURN node.managerName, score
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 0.2615291476249695}]

__Create nodes for all companies that filed a Form 13__

In [80]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""
# loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'managerParam': form13 })

In [81]:
kg.query("""
    MATCH (mgr:Manager) 
    RETURN count(mgr)
""")

[{'count(mgr)': 561}]

### Create relationships between managers and companies

- Match companies with managers based on data in the Form 13
- Create an `OWNS_STOCK_IN` relationship between the manager and the company

In [82]:
cypher = """
  MATCH (mgr:Manager {managerCik: $investmentParam.managerCik}), 
        (com:Company {cusip6: $investmentParam.cusip6})
  RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(cypher, params={ 
    'investmentParam': first_form13 
})

[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [83]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(cypher, params={ 'ownsParam': first_form13 })

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

In [84]:
kg.query("""
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->
        (com:Company {cusip6: $ownsParam.cusip6})
RETURN owns { .shares, .value }
""", params={ 'ownsParam': first_form13 })

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

__Create relationships between all of the managers who filed Form 13s and the company__

In [85]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter 
    }]->(com)
  ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
"""

#loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'ownsParam': form13 })

In [86]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

kg.query(cypher)

[{'investments': 561}]

## Create Address nodes

In [87]:
import usaddress

In [88]:
cypher = """
    MERGE (addr:Address {managerCik: $locationParam.managerCik})
        ON CREATE
            SET addr.address = $locationParam.managerAddress,
                addr.zipcode = $zipcode,
                addr.city = $city,
                addr.state = $state,
                addr.street = $street,
                addr.addressNumber = $addressNumber,
                addr.streetName = $streetName,
                addr.streetType = $streetType,
                addr.occupancyType = $occupancyType,
                addr.occupancyNumber = $occupancyNumber,
                addr.managerName = $locationParam.managerName
"""

parsed_address = usaddress.tag(first_form13['managerAddress'])[0]
kg.query(cypher, params={
    'locationParam': form13,
    'zipcode': parsed_address.get('Zipcode', ''),
    'city': parsed_address.get('PlaceName', ''),
    'state': parsed_address.get('StateName', ''),
    'street': parsed_address.get('AddressNumber', '') + ' ' + parsed_address.get('StreetName', '') + ' ' + parsed_address.get('StreetNamePostType', ''),
    'addressNumber': parsed_address.get('AddressNumber', ''),
    'streetName': parsed_address.get('StreetName', ''),
    'streetType': parsed_address.get('StreetNamePostType', ''),
    'occupancyType': parsed_address.get('OccupancyType', ''),
    'occupancyNumber': parsed_address.get('OccupancyIdentifier', ''),
})

[]

In [89]:
kg.query("""
    MATCH (addr:Address)
    RETURN addr LIMIT 1
""")

[{'addr': {'managerCik': '9634',
   'zipcode': '',
   'streetName': 'BAY',
   'streetType': 'STREET',
   'occupancyType': '',
   'address': 'PO BOX 2300, TULSA, OK, 74172',
   'occupancyNumber': '',
   'city': 'TORONTO',
   'street': '200 BAY STREET',
   'addressNumber': '200',
   'state': 'A6',
   'managerName': 'BOKF, NA'}}]

__Create a uniquness constraint to avoid duplicate manager addresses__

In [90]:
kg.query("""
    CREATE CONSTRAINT unique_manager_addr
    IF NOT EXISTS
    FOR (n:Address)
    REQUIRE n.managerCik IS UNIQUE
""")

[]

__Create a fulltext index of manager addresses to enable text search__

In [91]:
kg.query("""
CREATE FULLTEXT INDEX fullTextManagerAddress
    IF NOT EXISTS
    FOR (addr:Address)
    ON EACH [addr.address]
""")

[]

In [92]:
kg.query("""
    CALL db.index.fulltext.queryNodes("fullTextManagerAddress",
    "bay toronto") YIELD node, score
    RETURN node.managerCik, node.address, score
""")

[]

__Create nodes for all companies that filed a Form 13__

In [93]:
cypher = """
    MERGE (addr:Address {managerCik: $locationParam.managerCik})
        ON CREATE
            SET addr.address = $locationParam.managerAddress,
                addr.zipcode = $zipcode,
                addr.city = $city,
                addr.state = $state,
                addr.street = $street,
                addr.addressNumber = $addressNumber,
                addr.streetName = $streetName,
                addr.streetType = $streetType,
                addr.occupancyType = $occupancyType,
                addr.occupancyNumber = $occupancyNumber,
                addr.managerName = $locationParam.managerName
"""

for form13 in all_form13s:
    try:
        parsed_address = usaddress.tag(form13['managerAddress'])[0]
    except usaddress.RepeatedLabelError as e:
        None
    kg.query(cypher, params={
        'locationParam': form13,
        'zipcode': parsed_address.get('Zipcode', ''),
        'city': parsed_address.get('PlaceName', ''),
        'state': parsed_address.get('StateName', ''),
        'street': parsed_address.get('AddressNumber', '') + ' ' + parsed_address.get('StreetName', '') + ' ' + parsed_address.get('StreetNamePostType', ''),
        'addressNumber': parsed_address.get('AddressNumber', ''),
        'streetName': parsed_address.get('StreetName', ''),
        'streetType': parsed_address.get('StreetNamePostType', ''),
        'occupancyType': parsed_address.get('OccupancyType', ''),
        'occupancyNumber': parsed_address.get('OccupancyIdentifier', ''),
    })

In [94]:
kg.query("""
    MATCH (addr:Address)
    RETURN count(addr)
""")

[{'count(addr)': 561}]

### Create relationships between managers and adresses

- Create an `LOCATED_AT` relationship between the manager and the address

In [95]:
cypher = """
    MATCH (mgr:Manager {managerCik: $locationParam.managerCik}),
        (addr:Address {managerCik: $locationParam.managerCik})
    RETURN mgr.managerName, addr.address, $locationParam as location
"""

kg.query(cypher, params={'locationParam': first_form13})

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr.address': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'location': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [96]:
cypher = """
    MATCH (mgr:Manager {managerCik: $locationParam.managerCik}),
        (addr:Address {managerCik: $locationParam.managerCik})
    MERGE (mgr)-[:LOCATED_AT]->(addr)
    RETURN mgr.managerName, addr.address
"""

kg.query(cypher, params={'locationParam': first_form13})

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr.address': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5'}]

In [97]:
kg.query("""
    MATCH (mgr:Manager {managerCik: $locationParam.managerCik})
    -[:LOCATED_AT]->(addr:Address {managerCik: $locationParam.managerCik})
    RETURN mgr.managerName, addr.address
""", params={'locationParam': first_form13})

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr.address': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5'}]

__Create relationships between all of the managers who filed Form 13s and their addresses__

In [98]:
cypher = """
    MATCH (mgr:Manager {managerCik: $locationParam.managerCik}),
        (addr:Address {managerCik: $locationParam.managerCik})
    MERGE (mgr)-[:LOCATED_AT]->(addr)
    RETURN mgr.managerName, addr.address
"""
for form13 in all_form13s:
    kg.query(cypher, params={'locationParam': form13})

In [99]:
cypher = """
    MATCH (:Manager)-[loc:LOCATED_AT]->(:Address)
    RETURN count(loc) as locations
"""

kg.query(cypher)

[{'locations': 561}]

In [100]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {formId: STRING, cik: STRING,
cusip6: STRING, source: STRING} Company {cusip6: STRING,
companyName: STRING, cusip: STRING} Manager {managerCik:
STRING, managerName: STRING, managerAddress: STRING} Address
{managerCik: STRING, managerName: STRING, address: STRING,
zipcode: STRING, city: STRING, state: STRING, street:
STRING, addressNumber: STRING, streetName: STRING,
streetType: STRING, occupancyType: STRING, occupancyNumber:
STRING} Relationship properties: SECTION {f10kItem: STRING}
OWNS_STOCK_IN {reportCalendarOrQuarter: STRING, value:
FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)
(:Manager)-[:LOCATED_AT]->(:Address)


### Determining the number of investors

In [101]:
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
    """

chunk_rows = kg.query(cypher)
print(chunk_rows)

[{'chunkId': '0000950170-23-027948-item1-chunk0000'}]


In [102]:
chunk_first_row = chunk_rows[0]
print(chunk_first_row)

{'chunkId': '0000950170-23-027948-item1-chunk0000'}


In [103]:
ref_chunk_id = chunk_first_row['chunkId']
ref_chunk_id

'0000950170-23-027948-item1-chunk0000'

In [104]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [105]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [106]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName, 
        count(mgr.managerName) as numberOfinvestors 
LIMIT 1
"""

kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'com.companyName': 'NETAPP INC', 'numberOfinvestors': 561}]

### Use queries to build additional context for LLM

In [107]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares + 
        " shares of " + com.companyName + 
        " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """
kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'text': 'Royal Bank of Canada owns 842850 shares of NETAPP INC at a value of $64,395,000,000'},
 {'text': 'SHELTON CAPITAL MANAGEMENT owns 39124 shares of NETAPP INC at a value of $2,989,085,000'},
 {'text': 'PUBLIC EMPLOYEES RETIREMENT SYSTEM OF OHIO owns 106941 shares of NETAPP INC at a value of $8,170,000'},
 {'text': 'WILBANKS SMITH & THOMAS ASSET MANAGEMENT LLC owns 6617 shares of NETAPP INC at a value of $505,539,000'},
 {'text': 'DEPRINCE RACE & ZOLLO INC owns 320581 shares of NETAPP INC at a value of $24,492,389,000'},
 {'text': 'COMMERCE BANK owns 101422 shares of NETAPP INC at a value of $7,748,640,000'},
 {'text': 'D. E. Shaw & Co., Inc. owns 323440 shares of NETAPP INC at a value of $24,710,816,000'},
 {'text': 'EDMP, INC. owns 4698 shares of NETAPP INC at a value of $358,927,000'},
 {'text': 'NATIXIS ADVISORS, L.P. owns 40296 shares of NETAPP INC at a value of $3,079,000'},
 {'text': 'Smithfield Trust Co owns 215 shares of NETAPP INC at a value of $16,000'}]

In [108]:
results = kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})
print(textwrap.fill(results[0]['text'], 60))

Royal Bank of Canada owns 842850 shares of NETAPP INC at a
value of $64,395,000,000


In [109]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 67, offset: 66} for query: "UNWIND $data AS row MATCH (n:`Chunk`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'textEmbedding', row.embedding) YIELD node RETURN count(*)"


In [110]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com 
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.managerName + 
    " owns " + owns.shares + 
    " shares in " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) + "." 
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") + 
    "\n" + node.text AS text,
    score,
    { 
      source: node.source
    } as metadata
"""

In [111]:
vector_store_with_investment = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query,
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever_with_investments
)

In [112]:
question = "In a single sentence, tell me about Netapp."

In [113]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that provides customers the freedom to manage applications and data across hybrid multicloud environments. \n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [114]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that provides intelligent data management software for hybrid multicloud environments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [115]:
question = "In a single sentence, tell me about Netapp investors."

In [116]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'Netapp investors are diverse and include global enterprises, local businesses, and government installations who look to NetApp and its ecosystem of partners to maximize the business value of their IT and cloud investments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [117]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'Netapp investors include VANGUARD GROUP INC, BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

## Chatting with the SEC Knowledge Graph

In [118]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {formId: STRING, cik: STRING,
cusip6: STRING, source: STRING} Company {cusip6: STRING,
companyName: STRING, cusip: STRING} Manager {managerCik:
STRING, managerName: STRING, managerAddress: STRING} Address
{managerCik: STRING, managerName: STRING, address: STRING,
zipcode: STRING, city: STRING, state: STRING, street:
STRING, addressNumber: STRING, streetName: STRING,
streetType: STRING, occupancyType: STRING, occupancyNumber:
STRING} Relationship properties: SECTION {f10kItem: STRING}
OWNS_STOCK_IN {reportCalendarOrQuarter: STRING, value:
FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)
(:Manager)-[:LOCATED_AT]->(:Address)


__Check the address of a random manager__

In [119]:
kg.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr, addr
LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'},
  'addr': {'managerCik': '1000275',
   'zipcode': '',
   'streetName': 'BAY',
   'streetType': 'STREET',
   'occupancyType': '',
   'address': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'occupancyNumber': '',
   'city': 'TORONTO',
   'street': '200 BAY STREET',
   'addressNumber': '200',
   'state': 'A6',
   'managerName': 'Royal Bank of Canada'}}]

__Full text search for a manager named Royal Bank__

In [120]:
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank") YIELD node, score
  RETURN node.managerName, score LIMIT 1
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 3.7019896507263184}]

__Find location of Royal Bank__

In [121]:
kg.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.managerName, addr
""")

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr': {'managerCik': '1000275',
   'zipcode': '',
   'streetName': 'BAY',
   'streetType': 'STREET',
   'occupancyType': '',
   'address': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'occupancyNumber': '',
   'city': 'TORONTO',
   'street': '200 BAY STREET',
   'addressNumber': '200',
   'state': 'A6',
   'managerName': 'Royal Bank of Canada'}}]

__Determine which state has the most investment firms__

In [122]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'state': 'NY', 'numManagers': 58},
 {'state': 'CA', 'numManagers': 56},
 {'state': '', 'numManagers': 40},
 {'state': 'MA', 'numManagers': 32},
 {'state': 'PA', 'numManagers': 30},
 {'state': 'IL', 'numManagers': 30},
 {'state': 'FL', 'numManagers': 22},
 {'state': 'OH', 'numManagers': 19},
 {'state': 'TX', 'numManagers': 19},
 {'state': 'NJ', 'numManagers': 14}]

__What are the cities in California with the most investment firms?__

In [144]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'CA'
  RETURN address.city as city, count(address.city) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'city': 'San Francisco', 'numManagers': 6},
 {'city': 'Los Angeles', 'numManagers': 6},
 {'city': 'SAN FRANCISCO', 'numManagers': 5},
 {'city': 'LOS ANGELES', 'numManagers': 4},
 {'city': 'PASADENA', 'numManagers': 3},
 {'city': 'SACRAMENTO', 'numManagers': 2},
 {'city': 'SAN DIEGO', 'numManagers': 2},
 {'city': 'OAKLAND', 'numManagers': 2},
 {'city': 'BEVERLY HILLS', 'numManagers': 2},
 {'city': 'Foster City', 'numManagers': 2}]

__What are top investment firms in San Francisco?__

In [126]:
kg.query("""
  MATCH p=(mgr:Manager)-[:LOCATED_AT]->(address:Address),
         (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
         WHERE address.city = "San Francisco"
  RETURN mgr.managerName, sum(owns.value) as totalInvestmentValue
    ORDER BY totalInvestmentValue DESC
    LIMIT 10
""")

[{'mgr.managerName': 'SKBA CAPITAL MANAGEMENT LLC',
  'totalInvestmentValue': 13022380000.0},
 {'mgr.managerName': 'Legato Capital Management LLC',
  'totalInvestmentValue': 2991977000.0},
 {'mgr.managerName': 'INTACT INVESTMENT MANAGEMENT INC.',
  'totalInvestmentValue': 2681640000.0},
 {'mgr.managerName': 'Parallax Volatility Advisers, L.P.',
  'totalInvestmentValue': 1481320000.0},
 {'mgr.managerName': 'WETHERBY ASSET MANAGEMENT INC',
  'totalInvestmentValue': 848117000.0},
 {'mgr.managerName': 'Woodline Partners LP',
  'totalInvestmentValue': 327068000.0}]

### Writing Cypher with an LLM

- Using OpenAI's GPT 3.5 model
- Using Neo4j integration within LangChain called __`GraphCypherQAChain`__

In [154]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What management firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
The question is:
{question}"""

In [155]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [156]:
cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [157]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [158]:
prettyCypherChain("What management firms are in San Francisco?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
Full Context:
[{'mgr.managerName': 'WETHERBY ASSET MANAGEMENT INC'}, {'mgr.managerName': 'Legato Capital Management LLC'}, {'mgr.managerName': 'INTACT INVESTMENT MANAGEMENT INC.'}, {'mgr.managerName': 'Parallax Volatility Advisers, L.P.'}, {'mgr.managerName': 'Woodline Partners LP'}, {'mgr.managerName': 'SKBA CAPITAL MANAGEMENT LLC'}]

> Finished chain.
WETHERBY ASSET MANAGEMENT INC, Legato Capital Management
LLC, INTACT INVESTMENT MANAGEMENT INC., Parallax Volatility
Advisers, L.P., Woodline Partners LP, SKBA CAPITAL
MANAGEMENT LLC are management firms in San Francisco.


In [160]:
prettyCypherChain("What management firms are in Toronto?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'Toronto'
RETURN mgr.managerName
Full Context:
[{'mgr.managerName': 'Hillsdale Investment Management Inc.'}, {'mgr.managerName': 'State of Alaska, Department of Revenue'}, {'mgr.managerName': 'ONTARIO TEACHERS PENSION PLAN BOARD'}]

> Finished chain.
Hillsdale Investment Management Inc., ONTARIO TEACHERS
PENSION PLAN BOARD are management firms in Toronto.


In [161]:
prettyCypherChain("What investment firms are in Kanas City?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'Kansas City'
RETURN mgr.managerName
Full Context:
[{'mgr.managerName': 'UMB Bank, n.a.'}]

> Finished chain.
UMB Bank, n.a. is an investment firm located in Kansas City.
